In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable
import astropy.units as u
import sys
import pyneb as pn
from multiprocessing import Pool
import multiprocessing as mp
import math
from astropy.table import vstack
from multiprocessing.pool import ThreadPool

from concurrent.futures import ThreadPoolExecutor

### Import Refit data

In [ ]:
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}
galaxy = galdic[1]
infile = open(f"/home/habjan/jupfiles/data/{galaxy}_refitdata.fits",'rb')
data = Table.read(infile)
galaxy

### Set parameters

In [ ]:
mciters = 1000
snerr = 0

### PyNeb Diagnostics

In [ ]:
diags = pn.Diagnostics()

### NII Temperature and Denisty Uncertainty function

In [ ]:
def niitemperr(indata, iters):
    
    temperr = np.zeros(iters)
    denerr = np.zeros(iters)
    
    for j in range(iters):
        noise5754 = np.random.normal(0, indata['NII5754_FLUX_CORR_REFIT_ERR'])
        noise6583 = np.random.normal(0, indata['NII6583_FLUX_CORR_ERR'])
        noise6730 = np.random.normal(0, indata['SII6730_FLUX_CORR_ERR'])
        noise6716 = np.random.normal(0, indata['SII6716_FLUX_CORR_ERR'])

        temperr[j], denerr[j] = diags.getCrossTemDen(diag_tem = '[NII] 5755/6584', diag_den = '[SII] 6731/6716', 
                                                     value_tem = (indata['NII5754_FLUX_CORR_REFIT']+noise5754)/(indata['NII6583_FLUX_CORR']+noise6583), 
                                                     value_den = (indata['SII6730_FLUX_CORR']+noise6730)/(indata['SII6716_FLUX_CORR']+noise6716), 
                                                     guess_tem = 10**4)
            
    temperr = np.array([temperr[i] for i in range(len(temperr)) if np.isnan(temperr[i]) == False])
    denerr = np.array([denerr[i] for i in range(len(denerr)) if np.isnan(denerr[i]) == False])
    
    return np.std(temperr), np.std(denerr)

### SIII Temperature and Density Uncertainty Function

In [ ]:
def siiitemperr(indata, iters):
    
    stemperr = np.zeros(iters)
    otemperr = np.zeros(iters)
    denerr = np.zeros(iters)
    
    for j in range(iters):
        noise6312 = np.random.normal(0, indata['SIII6312_FLUX_CORR_REFIT_ERR'])
        noise9069 = np.random.normal(0, indata['SIII9068_FLUX_CORR_ERR'])
        noise6730 = np.random.normal(0, indata['SII6730_FLUX_CORR_ERR'])
        noise6716 = np.random.normal(0, indata['SII6716_FLUX_CORR_ERR'])

        stemperr[j], denerr[j] = diags.getCrossTemDen(diag_tem = '[SIII] 6312/9069', diag_den = '[SII] 6731/6716', 
                                                     value_tem = (indata['SIII6312_FLUX_CORR_REFIT']+noise6312)/(indata['SIII9068_FLUX_CORR']+noise9069), 
                                                     value_den = (indata['SII6730_FLUX_CORR']+noise6730)/(indata['SII6716_FLUX_CORR']+noise6716), 
                                                     guess_tem = 10**4)
        otemperr[j] = 0.7092 * stemperr[j] + 3609.9
            
    stemperr = np.array([stemperr[i] for i in range(len(stemperr)) if np.isnan(stemperr[i]) == False])
    otemperr = np.array([otemperr[i] for i in range(len(otemperr)) if np.isnan(otemperr[i]) == False])
    denerr = np.array([denerr[i] for i in range(len(denerr)) if np.isnan(denerr[i]) == False])
    
    return np.std(stemperr), np.std(otemperr), np.std(denerr)

### Temperature and Denisty Function

In [ ]:
def dentemp(indata, err, iters):
  
  tenii = np.zeros(len(indata))
  nenii = np.zeros(len(indata))
  teniierr = np.zeros(len(indata))
  neniierr = np.zeros(len(indata))
  for i in range(len(indata)):
        if indata[i]['BPT_NII'] == 0 and indata[i]['NII6583_FLUX_CORR'] > err * indata[i]['NII6583_FLUX_CORR_ERR'] and indata[i]['SII6730_FLUX_CORR'] > err * indata[i]['SII6730_FLUX_CORR_ERR'] and indata[i]['SII6716_FLUX_CORR'] > err * indata[i]['SII6716_FLUX_CORR_ERR'] and indata[i]['NII5754_FLUX_CORR_REFIT'] > err * indata[i]['NII5754_FLUX_CORR_REFIT_ERR']:
            tenii[i], nenii[i] = diags.getCrossTemDen(diag_tem = '[NII] 5755/6584', diag_den = '[SII] 6731/6716', 
                                      value_tem = indata[i]['NII5754_FLUX_CORR_REFIT']/indata[i]['NII6583_FLUX_CORR'], 
                                                value_den = indata[i]['SII6730_FLUX_CORR']/indata[i]['SII6716_FLUX_CORR'], 
                                                      guess_tem = 10**4)
            teniierr[i], neniierr[i] = niitemperr(indata[i], iters)
        else:
            tenii[i], nenii[i] = np.nan, np.nan
            teniierr[i], neniierr[i] = np.nan, np.nan

  tesiii = np.zeros(len(indata))
  nesiii = np.zeros(len(indata))
  tesiiierr = np.zeros(len(indata))
  nesiiierr = np.zeros(len(indata))
  teoiii = np.zeros(len(indata))
  teoiiierr = np.zeros(len(indata))
  for i in range(len(indata)):
    if indata[i]['BPT_NII'] == 0 and indata[i]['SIII9068_FLUX_CORR'] > err * indata[i]['SIII9068_FLUX_CORR_ERR'] and indata[i]['SII6730_FLUX_CORR'] > err * indata[i]['SII6730_FLUX_CORR_ERR'] and indata[i]['SII6716_FLUX_CORR'] > err * indata[i]['SII6716_FLUX_CORR_ERR'] and indata[i]['SIII6312_FLUX_CORR_REFIT'] > err * indata[i]['SIII6312_FLUX_CORR_REFIT_ERR']:
        tesiii[i], nesiii[i] = diags.getCrossTemDen(diag_tem = '[SIII] 6312/9069', diag_den = '[SII] 6731/6716', 
                                        value_tem = indata[i]['SIII6312_FLUX_CORR_REFIT']/indata[i]['SIII9068_FLUX_CORR'], 
                                              value_den = indata[i]['SII6730_FLUX_CORR']/indata[i]['SII6716_FLUX_CORR'], 
                                                    guess_tem = 10**4)
        tesiiierr[i], teoiiierr[i], nesiiierr[i] = siiitemperr(indata[i], iters)

        if tesiii[i] < 14000:
            teoiii[i] = 0.7092 * tesiii[i] + 3609.9
        else: 
            teoiii[i], teoiiierr[i] = np.nan, np.nan
    else: 
        tesiii[i], nesiii[i] = np.nan, np.nan
        tesiiierr[i], nesiiierr[i] = np.nan, np.nan

  indata.add_columns([tenii, tesiii, teoiii, nenii, nesiii, teniierr, tesiiierr, teoiiierr, neniierr, nesiiierr], 
                      names=('NII_TEMP', 'SIII_TEMP','OIII_TEMP', 'SII_DEN_NII', 'SII_DEN_SIII',
                            'NII_TEMP_ERR', 'SIII_TEMP_ERR','OIII_TEMP_ERR', 'SII_DEN_NII_ERR', 'SII_DEN_SIII_ERR'))

  return indata

# Multiprocessing of Temperature and Density Data

In [ ]:
if __name__ == "__main__":

    corenum = 32                            #chosen based of the number of cores
    batch = math.ceil(len(data)/corenum)     #batch determines the number of data points in each batched dataset
    datalist = [data[i:i+batch] for i in range(0, len(data), batch)] #make list of batched data
    
    pool = mp.Pool(processes = len(datalist))          #count processes are inititiated
    mplist = [pool.apply_async(dentemp, args = (d, snerr, mciters)) for d in datalist] #each batched dataset is assigned to a core 

In [ ]:
results = [mplist[i].get() for i in range(len(mplist))]      #Retrieve parallelized results

In [ ]:
#metdata.remove_columns(['MET_DIRECT', 'MET_DIRECT_OII7319', 'MET_DIRECT_OII7330', 'OII_ABUN', 'OIII_ABUN', 'MET_DIRECT_ERR', 'MET_DIRECT_OII7319_ERR', 'MET_DIRECT_OII7330_ERR', 'OII_ABUN_ERR', 'OIII_ABUN_ERR'])
#results = [metdata[i:i+batch] for i in range(0, len(metdata), batch)]

# Metallicity

### PyNeb atomic class information

In [ ]:
icf = pn.ICF()
O1 = pn.Atom('O', 1)
O2 = pn.Atom('O', 2)
O3 = pn.Atom('O', 3)
icf_list = ['Ial06_16']

### Monte Carlo Function for both [OII]7319,7330 derived metallicity

In [ ]:
def errboth(indata, iters):
    meterr = np.zeros(iters)
    ion2err = np.zeros(iters)
    ion3err = np.zeros(iters)
    
    for j in range(iters):
        
        noise7319 = np.random.normal(0, indata['OII7319_FLUX_CORR_REFIT_ERR'])
        noise7330 = np.random.normal(0, indata['OII7330_FLUX_CORR_REFIT_ERR'])
        noisetemp2 = np.random.normal(0, indata['NII_TEMP_ERR'])
        noiseden2 = np.random.normal(0, indata['SII_DEN_NII_ERR'])
        noise4861 = np.random.normal(0, indata['HB4861_FLUX_CORR_ERR'])
        noise5006 = np.random.normal(0, indata['OIII5006_FLUX_CORR_ERR'])
        noisetemp3 = np.random.normal(0, indata['OIII_TEMP_ERR'])
        noiseden3 = np.random.normal(0, indata['SII_DEN_SIII_ERR'])
        
        OII = O2.getIonAbundance(int_ratio = (indata['OII7319_FLUX_CORR_REFIT']+noise7319)+(indata['OII7330_FLUX_CORR_REFIT']+noise7330), 
                                     tem = (indata['NII_TEMP']+noisetemp2), 
                                     den= (indata['SII_DEN_NII']+noiseden2), to_eval='L(7320)+L(7330)', 
                                     Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
        OIII = O3.getIonAbundance(int_ratio = (indata['OIII5006_FLUX_CORR']+noise5006), 
                                      tem = (indata['OIII_TEMP']+noisetemp3), 
                                      den= (indata['SII_DEN_SIII']+noiseden3), to_eval='L(5007)', 
                                      Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
            
        abunlist = {'O2': OII, 'O3': OIII}
        abundance = icf.getElemAbundance(abunlist, icf_list)
        meterr[j] = 12 + np.log10(abundance['Ial06_16'])
        ion2err[j] = OII
        ion3err[j] = OIII
        
    meterr = np.array([meterr[i] for i in range(len(meterr)) if np.isnan(meterr[i]) == False])
    ion2err = np.array([ion2err[i] for i in range(len(ion2err)) if np.isnan(ion2err[i]) == False])
    ion3err = np.array([ion3err[i] for i in range(len(ion3err)) if np.isnan(ion3err[i]) == False])
    
    return np.std(meterr), np.std(ion2err), np.std(ion3err)

### Monte Carlo uncertainty for only [OII]7319 derived metallicity

In [ ]:
def err7319(indata, iters):
    met7319err = np.zeros(iters)
    
    for j in range(iters):
        
        noise7319 = np.random.normal(0, indata['OII7319_FLUX_CORR_REFIT_ERR'])
        noisetemp2 = np.random.normal(0, indata['NII_TEMP_ERR'])
        noiseden2 = np.random.normal(0, indata['SII_DEN_NII_ERR'])
        noise4861 = np.random.normal(0, indata['HB4861_FLUX_CORR_ERR'])
        noise5006 = np.random.normal(0, indata['OIII5006_FLUX_CORR_ERR'])
        noisetemp3 = np.random.normal(0, indata['OIII_TEMP_ERR'])
        noiseden3 = np.random.normal(0, indata['SII_DEN_SIII_ERR'])
        
        OII = O2.getIonAbundance(int_ratio = (indata['OII7319_FLUX_CORR_REFIT']+noise7319), 
                                     tem = (indata['NII_TEMP']+noisetemp2), 
                                     den= (indata['SII_DEN_NII']+noiseden2), to_eval='L(7320)', 
                                     Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
        OIII = O3.getIonAbundance(int_ratio = (indata['OIII5006_FLUX_CORR']+noise5006), 
                                      tem = (indata['OIII_TEMP']+noisetemp3), 
                                      den= (indata['SII_DEN_SIII']+noiseden3), to_eval='L(5007)', 
                                      Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
            
        abunlist = {'O2': OII, 'O3': OIII}
        abundance = icf.getElemAbundance(abunlist, icf_list)
        met7319err[j] = 12 + np.log10(abundance['Ial06_16'])
        
    met7319err = np.array([met7319err[i] for i in range(len(met7319err)) if np.isnan(met7319err[i]) == False])
    
    return np.std(met7319err)

### Monte Carlo Uncertainty for only [OII]7330 derived metallicity

In [ ]:
def err7330(indata, iters):
    met7330err = np.zeros(iters)
    
    for j in range(iters):
        
        noise7330 = np.random.normal(0, indata['OII7330_FLUX_CORR_REFIT_ERR'])
        noisetemp2 = np.random.normal(0, indata['NII_TEMP_ERR'])
        noiseden2 = np.random.normal(0, indata['SII_DEN_NII_ERR'])
        noise4861 = np.random.normal(0, indata['HB4861_FLUX_CORR_ERR'])
        noise5006 = np.random.normal(0, indata['OIII5006_FLUX_CORR_ERR'])
        noisetemp3 = np.random.normal(0, indata['OIII_TEMP_ERR'])
        noiseden3 = np.random.normal(0, indata['SII_DEN_SIII_ERR'])
        
        OII = O2.getIonAbundance(int_ratio = (indata['OII7330_FLUX_CORR_REFIT']+noise7330), 
                                     tem = (indata['NII_TEMP']+noisetemp2), 
                                     den= (indata['SII_DEN_NII']+noiseden2), to_eval='L(7330)', 
                                     Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
        OIII = O3.getIonAbundance(int_ratio = (indata['OIII5006_FLUX_CORR']+noise5006), 
                                      tem = (indata['OIII_TEMP']+noisetemp3), 
                                      den= (indata['SII_DEN_SIII']+noiseden3), to_eval='L(5007)', 
                                      Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
            
        abunlist = {'O2': OII, 'O3': OIII}
        abundance = icf.getElemAbundance(abunlist, icf_list)
        met7330err[j] = 12 + np.log10(abundance['Ial06_16'])
        
    met7330err = np.array([met7330err[i] for i in range(len(met7330err)) if np.isnan(met7330err[i]) == False])
    
    return np.std(met7330err)

### Metllicity Function

In [ ]:
def metal(indata, err, iters):

    met = np.zeros(len(indata))
    ion2 = np.zeros(len(indata))
    ion3 = np.zeros(len(indata))
    meterr = np.zeros(len(indata))
    ion2err = np.zeros(len(indata))
    ion3err = np.zeros(len(indata))
    
    for j in range(len(indata)):
        if indata[j]['OII7330_FLUX_CORR_REFIT'] > err * indata[j]['OII7330_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['OII7319_FLUX_CORR_REFIT'] > err * indata[j]['OII7319_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['HB4861_FLUX_CORR'] > err * indata[j]['HB4861_FLUX_CORR'] and \
        indata[j]['OIII5006_FLUX_CORR'] > err * indata[j]['OIII5006_FLUX_CORR_ERR']:
            OII = O2.getIonAbundance(int_ratio = (indata[j]['OII7330_FLUX_CORR_REFIT']+indata[j]['OII7319_FLUX_CORR_REFIT']), 
                                         tem = indata[j]['NII_TEMP'], den= indata[j]['SII_DEN_NII'], 
                                         to_eval='L(7320)+L(7330)', Hbeta = indata[j]['HB4861_FLUX_CORR'])
            OIII = O3.getIonAbundance(int_ratio = indata[j]['OIII5006_FLUX_CORR'], tem = indata[j]['OIII_TEMP'], 
                                      den= indata[j]['SII_DEN_SIII'], to_eval='L(5007)', 
                                      Hbeta = indata[j]['HB4861_FLUX_CORR'])
            
            abunlist = {'O2': OII, 'O3': OIII}
            abundance = icf.getElemAbundance(abunlist, icf_list)
            met[j] = 12 + np.log10(abundance['Ial06_16'])
            ion2[j] = OII
            ion3[j] = OIII
            meterr[j], ion2err[j], ion3err[j] = errboth(indata[j], iters)
        else: 
            met[j], ion2[j], ion3[j] = np.nan, np.nan, np.nan
            meterr[j], ion2err[j], ion3err[j] = np.nan, np.nan, np.nan

    met7319 = np.zeros(len(indata))
    met7319err = np.zeros(len(indata))
    
    for j in range(len(indata)):
        if indata[j]['OII7319_FLUX_CORR_REFIT'] > err * indata[j]['OII7319_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['HB4861_FLUX_CORR'] > err * indata[j]['HB4861_FLUX_CORR'] and \
        indata[j]['OIII5006_FLUX_CORR'] > err * indata[j]['OIII5006_FLUX_CORR_ERR']:
            OII = O2.getIonAbundance(int_ratio = (indata[j]['OII7319_FLUX_CORR_REFIT']), 
                                         tem = indata[j]['NII_TEMP'], den= indata[j]['SII_DEN_NII'], 
                                         to_eval='L(7320)', Hbeta = indata[j]['HB4861_FLUX_CORR'])
            OIII = O3.getIonAbundance(int_ratio = indata[j]['OIII5006_FLUX_CORR'], tem = indata[j]['OIII_TEMP'], 
                                      den= indata[j]['SII_DEN_SIII'], to_eval='L(5007)', 
                                      Hbeta = indata[j]['HB4861_FLUX_CORR'])
            
            abunlist = {'O2': OII, 'O3': OIII}
            abundance = icf.getElemAbundance(abunlist, icf_list)
            met7319[j] = 12 + np.log10(abundance['Ial06_16'])
            met7319err[j] = err7319(indata[j], iters)
        else: 
            met7319[j], met7319err[j] = np.nan, np.nan

    met7330 = np.zeros(len(indata))
    met7330err = np.zeros(len(indata))
    
    for j in range(len(indata)):
        if indata[j]['OII7330_FLUX_CORR_REFIT'] > err * indata[j]['OII7330_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['HB4861_FLUX_CORR'] > err * indata[j]['HB4861_FLUX_CORR'] and \
        indata[j]['OIII5006_FLUX_CORR'] > err * indata[j]['OIII5006_FLUX_CORR_ERR']:
            OII = O2.getIonAbundance(int_ratio = (indata[j]['OII7330_FLUX_CORR_REFIT']), 
                                         tem = indata[j]['NII_TEMP'], den= indata[j]['SII_DEN_NII'], 
                                         to_eval='L(7330)', Hbeta = indata[j]['HB4861_FLUX_CORR'])
            OIII = O3.getIonAbundance(int_ratio = indata[j]['OIII5006_FLUX_CORR'], tem = indata[j]['OIII_TEMP'], 
                                      den= indata[j]['SII_DEN_SIII'], to_eval='L(5007)', 
                                      Hbeta = indata[j]['HB4861_FLUX_CORR'])
            
            abunlist = {'O2': OII, 'O3': OIII}
            abundance = icf.getElemAbundance(abunlist, icf_list)
            met7330[j] = 12 + np.log10(abundance['Ial06_16'])
            met7330err[j] = err7330(indata[j], iters)
        else: 
            met7330[j], met7330err[j] = np.nan, np.nan

    indata.add_columns([met, met7319, met7330, ion2, ion3, meterr, met7319err, met7330err, ion2err, ion3err],
                     names=('MET_DIRECT', 'MET_DIRECT_OII7319', 'MET_DIRECT_OII7330', 'OII_ABUN', 'OIII_ABUN', 'MET_DIRECT_ERR', 'MET_DIRECT_OII7319_ERR', 'MET_DIRECT_OII7330_ERR', 'OII_ABUN_ERR', 'OIII_ABUN_ERR'))

    return indata

### Create multiprocessing objects

In [ ]:
if __name__ == "__main__":
    
    pool = mp.Pool(processes = len(results))          #count processes are inititiated
    mplist2 = [pool.apply_async(metal, args = (r, snerr, 5000)) for r in results]

### Get results from multiprocessing objects

In [ ]:
results2 = [mplist2[i].get() for i in range(len(mplist2))]
metdata = vstack(results2)

# Plot metallicity Results

### Convert deprojected distancs to kpc

In [ ]:
distdic = {'NGC4254': 13.1 , 'NGC4535': 16.60  , 'NGC3351': 10.0, 'NGC2835': 10.4, 'NGC0628':7.5}
dist = distdic[galaxy]
kpc = data['deproj_dist'] / 206265 * (dist * 1000)
nkpc = np.array([kpc[i] for i in range(len(kpc))])
seq = np.linspace(-3, 15, num = 20)

### Best fit for [OII]7319,7330 metallicity

In [ ]:
dirc = np.array([metdata[i]['MET_DIRECT'] for i in range(len(metdata)) if np.isnan(metdata[i]['MET_DIRECT']) == False])
direct = np.polyfit(nkpc[np.where(np.isnan(metdata['MET_DIRECT']) == False)], dirc, 1)
dirfunc = np.poly1d(direct)
np.std(dirc)

### Best fit for [OII]7319 metallicity

In [ ]:
dirc7319 = np.array([metdata[i]['MET_DIRECT_OII7319'] for i in range(len(metdata)) if np.isnan(metdata[i]['MET_DIRECT_OII7319']) == False])
direct7319 = np.polyfit(nkpc[np.where(np.isnan(metdata['MET_DIRECT_OII7319']) == False)], dirc7319, 1)
dirfunc7319 = np.poly1d(direct7319)
np.std(dirc7319)

### Best fit for [OII]7330 metallicity

In [ ]:
dirc7330 = np.array([metdata[i]['MET_DIRECT_OII7330'] for i in range(len(metdata)) if np.isnan(metdata[i]['MET_DIRECT_OII7330']) == False])
direct7330 = np.polyfit(nkpc[np.where(np.isnan(metdata['MET_DIRECT_OII7330']) == False)], dirc7330, 1)
dirfunc7330 = np.poly1d(direct7330)
np.std(dirc7330)

### Best fit for Strong Line metallicity

In [ ]:
s = np.array([metdata[i]['met_scal'] for i in range(len(metdata)) if np.isnan(metdata[i]['met_scal']) == False])
scal = np.polyfit(nkpc[np.where(np.isnan(metdata['met_scal']) == False)], s, 1)
scalfunc = np.poly1d(scal)
np.std(s)

### Plot results and best fit lines

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 4)
ax1 = fig.add_subplot(111)

ax1.scatter(kpc, metdata['met_scal'], s=10, c='orange', marker= '.', alpha=0.7, label='Scal-PG16')
ax1.plot(seq, scalfunc(seq), c='orange',  linestyle="--", label=f'Scal-PG16: y = {round(scalfunc[1],3)}x + {round(scalfunc[0],3)}')

ax1.scatter(kpc, metdata['MET_DIRECT'], s=30, c='blue', marker= '.', label=f'{len(dirc)} Directly Derived Z from both')
ax1.errorbar(kpc, metdata['MET_DIRECT'], yerr = metdata['MET_DIRECT_ERR'], c='blue',alpha=0.35, fmt=".", markersize=8, capsize=5)
ax1.plot(seq, dirfunc(seq), c='blue', linestyle="--", label=f'Directly Derived: y = {round(dirfunc[1],3)}x + {round(dirfunc[0],3)}')


#ax1.scatter(kpc, metdata['MET_DIRECT_OII7330'], s=20, c='k', marker= '*', label=f'{len(dirc7330)} Directly Derived Z from [O II]7330')
#ax1.errorbar(kpc, metdata['MET_DIRECT_OII7330'], yerr = metdata['MET_DIRECT_OII7330_ERR'], c='k',alpha=0.35, fmt=".", markersize=8, capsize=5)
#ax1.plot(seq, dirfunc7330(seq), c='k', linestyle="--", label=f'[OII]7330 Directly Derived: y = {round(dirfunc7330[1],3)}x + {round(dirfunc7330[0],3)}')


#ax1.scatter(kpc, metdata['MET_DIRECT_OII7319'], s=20, c='red', marker= 'x', label=f'{len(dirc7319)} Directly Derived Z from [O II]7319')
#ax1.errorbar(kpc, metdata['MET_DIRECT_OII7319'], yerr = metdata['MET_DIRECT_OII7319_ERR'], c='red',alpha=0.35, fmt=".", markersize=8, capsize=5)
#ax1.plot(seq, dirfunc7319(seq), c='red', linestyle="--", label=f'[OII]7320 Directly Derived: y = {round(dirfunc7319[1],3)}x + {round(dirfunc7319[0],3)}')

minm = np.min(metdata['MET_DIRECT'][np.where(np.isnan(metdata['MET_DIRECT']) == False)])
maxm = np.max(metdata['MET_DIRECT'][np.where(np.isnan(metdata['MET_DIRECT']) == False)])
minmerr = np.min(metdata['MET_DIRECT_ERR'][np.where(np.isnan(metdata['MET_DIRECT_ERR']) == False)])
maxmerr = np.max(metdata['MET_DIRECT_ERR'][np.where(np.isnan(metdata['MET_DIRECT_ERR']) == False)])
plt.xlim([np.min(kpc) - np.min(kpc)*0.01, np.max(kpc) + np.max(kpc)*0.01])
#plt.xlim(2, 8.5)
plt.ylim([minm - maxmerr*2, maxm + maxmerr*3])

plt.xlabel('Galactocentric Radius (kpc)', fontsize=15)
plt.ylabel('12 + log(O/H)', fontsize=15)
metnum = len(np.where(np.isnan(metdata['MET_DIRECT']) == False)[0])
plt.title(f'Metallicity calculations in {galaxy}')
plt.legend(bbox_to_anchor=(1.6, 0.5), fontsize="9",loc='center right');
plt.show()

### Plot a single metallicity versus the strong line metallicities

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 4)
ax1 = fig.add_subplot(111)

ax1.scatter(kpc, metdata['met_scal'], s=10, c='orange', marker= '.', alpha=0.7, label='Scal-PG16')
ax1.plot(seq, scalfunc(seq), c='orange',  linestyle="--", label=f'Scal-PG16: y = {round(scalfunc[1],3)}x + {round(scalfunc[0],3)}')


ax1.scatter(kpc, metdata['MET_DIRECT_OII7319'], s=20, c='red', marker= 'x', label=f'{len(dirc7319)} Directly Derived Z from [O II]7319')
ax1.errorbar(kpc, metdata['MET_DIRECT_OII7319'], yerr = metdata['MET_DIRECT_OII7319_ERR'], c='red',alpha=0.35, fmt=".", markersize=8, capsize=5)
ax1.plot(seq, dirfunc7319(seq), c='red', linestyle="--", label=f'[OII]7320 Directly Derived: y = {round(dirfunc7319[1],3)}x + {round(dirfunc7319[0],3)}')

minm = np.min(metdata['MET_DIRECT_OII7319'][np.where(np.isnan(metdata['MET_DIRECT_OII7319']) == False)])
maxm = np.max(metdata['MET_DIRECT_OII7319'][np.where(np.isnan(metdata['MET_DIRECT_OII7319']) == False)])
minmerr = np.min(metdata['MET_DIRECT_OII7319_ERR'][np.where(np.isnan(metdata['MET_DIRECT_OII7319_ERR']) == False)])
maxmerr = np.max(metdata['MET_DIRECT_OII7319_ERR'][np.where(np.isnan(metdata['MET_DIRECT_OII7319_ERR']) == False)])
plt.xlim([np.min(kpc) - np.min(kpc)*0.01, np.max(kpc) + np.max(kpc)*0.01])
#plt.xlim(2, 8.5)
plt.ylim([minm - maxmerr*1.2, maxm + maxmerr*0.9])

plt.xlabel('Galactocentric Radius (kpc)', fontsize=15)
plt.ylabel('12 + log(O/H)', fontsize=15)
metnum = len(np.where(np.isnan(metdata['MET_DIRECT']) == False)[0])
plt.title(f'Metallicity calculations in {galaxy}')
plt.legend(bbox_to_anchor=(1.6, 0.5), fontsize="9",loc='center right');
plt.show()

# Save Metallicity Results

In [ ]:
metdata.write(f'/home/habjan/jupfiles/data/{galaxy}_physdata.fits', overwrite=True)  #, overwrite=True